# DAMR
Hi, here is the code of DAMR(Dynamic Ajacency Matrix Representation) 
for data imputation in COVID-19 dataset.

# 1. Import library

In [1]:
#1.Import library
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import networkx as nx
from networkx.algorithms import bipartite
import copy
from sklearn.preprocessing import StandardScaler
import scipy.special as special
import scipy.sparse as sp
import pickle
import os
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

import tensorflow as tf
#import tensorflow_addons as tfa
import keras
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Dense, GRU, Input
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input, Dropout, Dense, LeakyReLU, GRU, Concatenate, Reshape, Softmax, Attention
from tensorflow.keras import activations
from tensorflow.keras.models import Model
#from keras.layers import LeakyReLU
from spektral.layers import  GCSConv, DiffusionConv, GATConv, ARMAConv, GCNConv
from tensorflow.keras.regularizers import l2
#from my own library import Generate_missing and other models
from model import * #generate_miss(data,perc),output:data after generate missing.
import FancyImpute
from FancyImpute import *
import torch
#Bidirectional LSTM
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Bidirectional
from BiLSTM import BiLSTM_3D

# 2. Data Pre-processing of COVID dataset

In [2]:
#Define hyperparameter
Miss_perc=0.7
Split_perc1=0.8
Split_perc2=0.1


In [3]:
#Generate Missing 3D.
path='Data/COVID/raw'
COVID_features=Csv2Tensor(path)
COVID_features=COVID_features[:503,:,:]
COVID_features=MinMax3D(COVID_features)#(503,3,7)
COVID_features_train=COVID_features[:int(Split_perc1*COVID_features.shape[0]),:,:]
print(COVID_features_train.shape)#(402, 3, 7)
COVID_features_val=COVID_features[int(Split_perc1*COVID_features.shape[0]):int(Split_perc1*COVID_features.shape[0])+int(Split_perc2*COVID_features.shape[0]),:,:]
print(COVID_features_val.shape)#(50, 3, 7)
COVID_features_test=COVID_features[int(Split_perc1*COVID_features.shape[0])+int(Split_perc2*COVID_features.shape[0]):,:,:]
print(COVID_features_test.shape)#(51, 3, 7)

#The mask and missing data has been created with a given miss percentage.
#It will be used for baseline and for DAMR
COVID_features_train_miss,COVID_features_train_mask=generate_miss_3D(COVID_features_train,Miss_perc)
COVID_features_val_miss,COVID_features_val_mask=generate_miss_3D(COVID_features_val,Miss_perc)
COVID_features_test_miss,COVID_features_test_mask=generate_miss_3D(COVID_features_test,Miss_perc)
COVID_features_miss,COVID_features_mask=generate_miss_3D(COVID_features,Miss_perc)


()
()
()
()
()
()
()
(402, 3, 7)
(50, 3, 7)
(51, 3, 7)


In [4]:
#BiLSTM_finish
COVID_features_miss,COVID_features_mask=generate_miss_3D(COVID_features,0.1)
BiLSTM_rmse,BiLSTM_mae,BiLSTM_mape,BiLSTM_mre,BiLSTM_err=BiLSTM_3D(COVID_features_miss,COVID_features,COVID_features_mask,0)
print("BiLSTM_rmse,",BiLSTM_rmse)
print("BiLSTM_mae",BiLSTM_mae)
print("BiLSTM_mape",BiLSTM_mape)
print("BiLSTM_mre",BiLSTM_mre)
print("BiLSTM_err",BiLSTM_err)

(402, 1, 3) (402, 3) (100, 1, 3) (100, 3)
Epoch 1/80
1/1 - 9s - loss: 0.4458 - val_loss: 0.9436
Epoch 2/80
1/1 - 0s - loss: 0.4434 - val_loss: 0.9543
Epoch 3/80
1/1 - 0s - loss: 0.4411 - val_loss: 0.9630
Epoch 4/80
1/1 - 0s - loss: 0.4389 - val_loss: 0.9698
Epoch 5/80
1/1 - 0s - loss: 0.4368 - val_loss: 0.9750
Epoch 6/80
1/1 - 0s - loss: 0.4348 - val_loss: 0.9784
Epoch 7/80
1/1 - 0s - loss: 0.4329 - val_loss: 0.9804
Epoch 8/80
1/1 - 0s - loss: 0.4309 - val_loss: 0.9810
Epoch 9/80
1/1 - 0s - loss: 0.4290 - val_loss: 0.9803
Epoch 10/80
1/1 - 0s - loss: 0.4270 - val_loss: 0.9785
Epoch 11/80
1/1 - 0s - loss: 0.4250 - val_loss: 0.9757
Epoch 12/80
1/1 - 0s - loss: 0.4230 - val_loss: 0.9722
Epoch 13/80
1/1 - 0s - loss: 0.4210 - val_loss: 0.9679
Epoch 14/80
1/1 - 0s - loss: 0.4190 - val_loss: 0.9630
Epoch 15/80
1/1 - 0s - loss: 0.4169 - val_loss: 0.9577
Epoch 16/80
1/1 - 0s - loss: 0.4148 - val_loss: 0.9519
Epoch 17/80
1/1 - 0s - loss: 0.4127 - val_loss: 0.9458
Epoch 18/80
1/1 - 0s - loss: 0.4

In [5]:
#Run baseline(KNN,MICE,MF,MEAN,SLIDING)#Return:RMSE,MAE,MAPE,MRE
#For the model of FancyImpute,we just need to test.
KNN_RMSE,KNN_MAE,KNN_MAPE,KNN_MRE=KNN_impute_3D(COVID_features_test_miss,COVID_features_test,COVID_features_test_mask)
MICE_RMSE,MICE_MAE,MICE_MAPE,MICE_MRE=MICE_impute_3D(COVID_features_test_miss,COVID_features_test,COVID_features_test_mask)
MF_RMSE,MF_MAE,MF_MAPE,MF_MRE=MF_impute_3D(COVID_features_test_miss,COVID_features_test,COVID_features_test_mask)
MEAN_RMSE,MEAN_MAE,MEAN_MAPE,MEAN_MRE=MEAN_impute_3D(COVID_features_test_miss,COVID_features_test,COVID_features_test_mask)
Sliding_RMSE,Sliding_MAE,Sliding_MAPE,Sliding_MRE,Sliding_out=Sliding_impute_3D(COVID_features_test_miss,COVID_features_test,COVID_features_test_mask,6)
print("KNN_RMSE",KNN_RMSE,"KNN_MAE",KNN_MAE,"KNN_MAPE",KNN_MAPE,"KNN_MRE",KNN_MRE)
print("MICE_RMSE",MICE_RMSE,"MICE_MAE",MICE_MAE,"MICE_MAPE",MICE_MAPE,"MICE_MRE",MICE_MRE)
print("MF_RMSE",MF_RMSE,"MF_MAE",MF_MAE,"MF_MAPE",MF_MAPE,"MF_MRE",MF_MRE)
print("MEAN_RMSE",MEAN_RMSE,"MEAN_MAE",MEAN_MAE,"MEAN_MAPE",MEAN_MAPE,"MEAN_MRE",MEAN_MRE)
print("Sliding_RMSE",Sliding_RMSE,"Sliding_MAE",Sliding_MAE,"Sliding_MAPE",Sliding_MAPE,"Sliding_MRE",Sliding_MRE)

Imputing row 1/51 with 3 missing, elapsed time: 0.002
[KNN] Warning: 69/153 still missing after imputation, replacing with 0
Imputing row 1/51 with 2 missing, elapsed time: 0.003
[KNN] Warning: 42/153 still missing after imputation, replacing with 0
Imputing row 1/51 with 1 missing, elapsed time: 0.004
[KNN] Warning: 51/153 still missing after imputation, replacing with 0
Imputing row 1/51 with 2 missing, elapsed time: 0.008
[KNN] Warning: 69/153 still missing after imputation, replacing with 0
Imputing row 1/51 with 3 missing, elapsed time: 0.002
[KNN] Warning: 66/153 still missing after imputation, replacing with 0
Imputing row 1/51 with 2 missing, elapsed time: 0.004
[KNN] Warning: 57/153 still missing after imputation, replacing with 0
Imputing row 1/51 with 1 missing, elapsed time: 0.007
[KNN] Warning: 54/153 still missing after imputation, replacing with 0
                                     CVXPY                                     
                                     v1.2.0  

(CVXPY) May 29 10:54:21 AM: Applying reduction Dcp2Cone
(CVXPY) May 29 10:54:21 AM: Applying reduction CvxAttr2Constr
(CVXPY) May 29 10:54:21 AM: Applying reduction ConeMatrixStuffing
(CVXPY) May 29 10:54:21 AM: Applying reduction CVXOPT
(CVXPY) May 29 10:54:21 AM: Finished problem compilation (took 1.967e-01 seconds).
-------------------------------------------------------------------------------
                                Numerical solver                               
-------------------------------------------------------------------------------
(CVXPY) May 29 10:54:21 AM: Invoking solver CVXOPT  to obtain a solution.
     pcost       dcost       gap    pres   dres   k/t
 0:  0.0000e+00 -1.5300e-02  2e+03  9e+00  4e+00  1e+00
 1:  1.2488e+01  1.3890e+01  4e+02  3e+00  1e+00  2e+00
 2:  8.7370e+00  9.4894e+00  1e+02  8e-01  4e-01  8e-01
 3:  9.2519e+00  9.3349e+00  9e+00  8e-02  4e-02  9e-02
 4:  9.3453e+00  9.3569e+00  2e+00  2e-02  9e-03  1e-02
 5:  9.3479e+00  9.3492e+00  4e

(CVXPY) May 29 10:55:04 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) May 29 10:55:04 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) May 29 10:55:04 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) May 29 10:55:04 AM: Compiling problem (target solver=CVXOPT).
(CVXPY) May 29 10:55:04 AM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing -> CVXOPT
(CVXPY) May 29 10:55:04 AM: Applying reduction Dcp2Cone
(CVXPY) May 29 10:55:04 AM: Applying reduction CvxAttr2Constr
(CVXPY) May 29 10:55:04 AM: Applying reduction ConeMatrixStuffing
(CVXPY) May 29 10:55:04 AM: Applying reduction CVXOPT
(CVX

# Run Baseline

In [6]:
##Lags for variable 1 ,modify input variable here!
COVID_X1_all=COVID_features[:,0,:]
#Transform the shape into Bidirectional problems
COVID_X1_all=series_to_supervised(COVID_X1_all,15,16)
COVID_Y1_all=COVID_X1_all.iloc[:,15*7:15*7+7]
COVID_X1_all_1=COVID_X1_all.iloc[:,:15*7]
COVID_X1_all_2=COVID_X1_all.iloc[:,15*7+7:]
COVID_X1_all=np.hstack((COVID_X1_all_1,COVID_X1_all_2))
COVID_X1_all=np.array(COVID_X1_all)
COVID_X1_all=np.reshape(COVID_X1_all,(len(COVID_X1_all),COVID_features.shape[2],30))
#Input 1->Groundtruth
COVID_features=COVID_features[15:-15,:,:]#(473, 3, 7)
print(COVID_features.shape)
#Input2->Groundtruth
print(COVID_X1_all.shape)#(473,7,30)
COVID_Y1_all=np.array(COVID_Y1_all)
COVID_Y1_all=np.reshape(COVID_Y1_all,(len(COVID_Y1_all),7,1))
#Input3->Groundtruth
Adj_all = np.arange(473*7*7).reshape(473,7,7)#(473,7,7)
#Output1->Groundtruth
print(COVID_Y1_all.shape)#(473, 7, 1)

(473, 3, 7)
(473, 7, 30)
(473, 7, 1)


In [7]:
##Input and output in DAMR!
# print(COVID_features_aftslid.shape)#473*3*7
# print(COVID_X1_all_aftslid.shape)#473*7*30
# print(COVID_Y1_all.shape)#(473, 7, 1)
# print(Adj_dist.shape)#7*7

#The mask and missing data has been created with a given miss percentage.
#It will be used for baseline and for DAMR
#Generate Missing 3D.
COVID_features=Csv2Tensor(path)
COVID_features=COVID_features[:503,:,:]
COVID_features=MinMax3D(COVID_features)#(503,3,7)->Groundtruth
COVID_features_train=COVID_features[:int(Split_perc1*COVID_features.shape[0]),:,:]
print(COVID_features_train.shape)#(402, 3, 7)
COVID_features_val=COVID_features[int(Split_perc1*COVID_features.shape[0]):int(Split_perc1*COVID_features.shape[0])+int(Split_perc2*COVID_features.shape[0]),:,:]
print(COVID_features_val.shape)#(50, 3, 7)
COVID_features_test=COVID_features[int(Split_perc1*COVID_features.shape[0])+int(Split_perc2*COVID_features.shape[0]):,:,:]
print(COVID_features_test.shape)#(51, 3, 7)

#
COVID_features_train_miss,COVID_features_train_mask=generate_miss_3D(COVID_features_train,Miss_perc)#(402, 3, 7)
COVID_features_val_miss,COVID_features_val_mask=generate_miss_3D(COVID_features_val,Miss_perc)#(50, 3, 7)
COVID_features_test_miss,COVID_features_test_mask=generate_miss_3D(COVID_features_test,Miss_perc)#(51, 3, 7)
COVID_features_miss,COVID_features_mask=generate_miss_3D(COVID_features,Miss_perc)

Sliding_RMSE,Sliding_MAE,Sliding_MAPE,Sliding_MRE,Sliding_out=Sliding_impute_3D(COVID_features_miss,COVID_features,COVID_features_mask,6)#(503,3,7)




()
()
()
()
()
()
()
(402, 3, 7)
(50, 3, 7)
(51, 3, 7)


In [8]:
#Create lags for the first variable.
#Extract the first variable
lags=30
var=0
Sliding_out_var1=Sliding_out[:,var,:]
Sliding_out_var1_sup=series_to_supervised(Sliding_out_var1,lags,1)
Sliding_out_var1_sup=Sliding_out_var1_sup.iloc[:,:lags*Sliding_out.shape[2]]
print(Sliding_out_var1_sup)
#Input1: 473*7*30
COVID_X1_all_aftslid=np.array(Sliding_out_var1_sup).reshape(len(Sliding_out_var1_sup),COVID_features.shape[2],lags)
#Input2: 473*3*7
COVID_features_aftslid=Sliding_out[30:,:,:]
#Input3: Adj: 473*7*7(Initialization)
Adj_all = np.arange(len(COVID_features_aftslid)*COVID_features_aftslid.shape[2]*COVID_features_aftslid.shape[2]).reshape(len(COVID_features_aftslid),COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2])#(473,7,7)
#Output:Groundtruth of the first variable->
COVID_Y1_all=COVID_features[30:,:,:]
COVID_Y1_all=COVID_Y1_all[:,var,:]
COVID_Y1_all=COVID_Y1_all.reshape(COVID_Y1_all.shape[0],COVID_Y1_all.shape[1],1)
#
print(COVID_X1_all_aftslid.shape,COVID_features_aftslid.shape,Adj_all.shape,COVID_Y1_all.shape)

     var1(t-30)  var2(t-30)  var3(t-30)  var4(t-30)  var5(t-30)  var6(t-30)  \
30    -0.999953   -1.000000   -1.000000   -1.000000   -0.999984   -0.999874   
31    -0.999953   -1.000000   -0.999841   -0.999942   -0.999961   -0.999812   
32    -1.000000   -1.000000   -0.999702   -0.999942   -0.999943   -0.999812   
33    -0.999970   -0.999999   -0.999561   -0.999883   -1.000000   -1.000000   
34    -0.999918   -0.999999   -1.000000   -0.999841   -0.999968   -0.999874   
..          ...         ...         ...         ...         ...         ...   
498    0.930432    0.866301    0.933753    0.780069    0.844052    0.604353   
499    0.944149    0.878392    0.930000    0.781885    0.870060    0.667740   
500    0.932800    0.911119    0.943566    0.783638    0.853848    0.689613   
501    0.936598    0.884859    0.947691    0.783677    0.855388    0.703309   
502    0.938415    0.919484    0.951197    0.785153    0.863110    0.679008   

     var7(t-30)  var1(t-29)  var2(t-29)  var3(t-29)

In [9]:
type(Sliding_out_var1_sup)

pandas.core.frame.DataFrame

# 3.Dynamic Adjacency matrix layer

In [10]:
#3.1 Distance-based adjacency matrix 
#Country: BZ CL ID IR MX PR SA.
COVID_lat=pd.DataFrame([-14.235,-35.6751,20.5937,32.4279,23.6345,-9.19,30.5595])
COVID_long=pd.DataFrame([-51.9253,-71.543,78.9629,53.688,-102.5528,-75.0152,22.9375])
Adj_dist=adj_dist(COVID_lat,COVID_long)#7*7

In [11]:
data1=pd.DataFrame([[-51.9253,-71.543,78.9629],[53.688,-102.5528,-75.0152]])
data2=pd.DataFrame([[-31.9253,-41.543,38.9629],[43.688,62.5528,65.0152]])
Global_MeanMI(data1,data2,2)
data1=pd.DataFrame(data1)
data2=pd.DataFrame(data2)
data1=np.array(data1).reshape(-1)
data2=np.array(data2).reshape(-1)

In [12]:
#3.2 Mean MI-based adjacency matrix
print(COVID_features_aftslid.shape)
MeanMI=[]
for i in range(COVID_features_aftslid.shape[2]):
    for j in range(COVID_features_aftslid.shape[2]):
        MeanMI.append(Global_MeanMI(COVID_features_aftslid[:,:,i],
                                    COVID_features_aftslid[:,:,j],2))
print(np.array(MeanMI).shape)
MeanMI=np.array(MeanMI).reshape(COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2])
print(MeanMI.shape)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled1 = scaler.fit_transform(MeanMI)
MeanMI=scaled1#shape:7*7
print(MeanMI)

(473, 3, 7)
(49,)
(7, 7)
[[1.         0.08090689 0.65486353 0.68290363 0.51826061 0.8254351
  0.22158466]
 [0.         1.         0.         0.         0.         0.
  0.        ]
 [0.63804715 0.20008227 1.         0.45934096 0.67730467 0.55645918
  0.37087971]
 [0.6876565  0.         0.43447838 1.         0.3421395  0.79876665
  0.28637826]
 [0.51707594 0.25166251 0.66276323 0.39301962 1.         0.45187307
  0.20021864]
 [0.83417338 0.04536679 0.5597045  0.79332129 0.43129736 1.
  0.20980373]
 [0.29065932 0.30301326 0.37846483 0.35761106 0.21735803 0.29257376
  1.        ]]


In [13]:
#3.3 CONCAT MI-based adjacency matrix
print(COVID_features_aftslid.shape)
ConcatMI=[]
for i in range(COVID_features_aftslid.shape[2]):
    for j in range(COVID_features_aftslid.shape[2]):
        ConcatMI.append(Global_MeanMI(COVID_features_aftslid[:,:,i],
                                    COVID_features_aftslid[:,:,j],2))
ConcatMI=np.array(ConcatMI).reshape(COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2])
scaler = MinMaxScaler(feature_range=(0, 1))
scaled1 = scaler.fit_transform(ConcatMI)
ConcatMI=scaled1#shape:7*7
print(ConcatMI)

(473, 3, 7)
[[1.         0.08090689 0.65486353 0.68290363 0.51826061 0.8254351
  0.22158466]
 [0.         1.         0.         0.         0.         0.
  0.        ]
 [0.63804715 0.20008227 1.         0.45934096 0.67730467 0.55645918
  0.37087971]
 [0.6876565  0.         0.43447838 1.         0.3421395  0.79876665
  0.28637826]
 [0.51707594 0.25166251 0.66276323 0.39301962 1.         0.45187307
  0.20021864]
 [0.83417338 0.04536679 0.5597045  0.79332129 0.43129736 1.
  0.20980373]
 [0.29065932 0.30301326 0.37846483 0.35761106 0.21735803 0.29257376
  1.        ]]


In [14]:
#3.4 Single Feature MI-based adjacency matrix
COVID_features_aftslid.shape
FeatureMI=[]
for i in range(COVID_features_aftslid.shape[2]):
    for j in range(COVID_features_aftslid.shape[2]):
        FeatureMI.append(Global_EachMI(COVID_features_aftslid[:,:,i],
                                    COVID_features_aftslid[:,:,j],2))
#For the first feature,if choose the second feature, modify here!!
FeatureMI=np.array(FeatureMI)[:,2]
FeatureMI=np.array(FeatureMI).reshape(COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2])
scaler = MinMaxScaler(feature_range=(0, 1))
scaled1 = scaler.fit_transform(FeatureMI)
FeatureMI=scaled1#shape:7*7
#MeanMI,ConcatMI,FeatureMI

In [15]:
#3.5 Mean MI-based dynamic adjacency matrix
print(COVID_features_aftslid.shape)
#Index all of the rows and columns
DyMeanMI=[]
for k in range(COVID_features_aftslid.shape[0]):
    for i in range(COVID_features_aftslid.shape[2]):
        for j in range(COVID_features_aftslid.shape[2]):
            DyMeanMI.append(Global_MeanMI(COVID_features_aftslid[:k,:,i],
                                    COVID_features_aftslid[:k,:,j],2))
DyMeanMI=np.array(DyMeanMI).reshape(COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[0])
DyMeanMI=DyMeanMI.transpose(2, 0, 1)
from fancyimpute import SimpleFill, NuclearNormMinimization, SoftImpute, BiScaler
for i in range(DyMeanMI.shape[0]):
    MEAN_imputer = SimpleFill()
    DyMeanMI[i,:,:]=MEAN_imputer.fit_transform(DyMeanMI[i,:,:])
print(DyMeanMI)#Here DyMeanMI is done

(473, 3, 7)


/Users/macbook/Desktop/PHD/CIKM/DAMR_CODE/model.py:67: RuntimeWarning: invalid value encountered in true_divide
  c_normalized = c / float(np.sum(c))


[[[0.57013036 0.91545221 0.57364931 ... 0.41844911 0.24771303 0.26333598]
  [0.70517372 0.55803727 0.48686508 ... 0.30034378 0.29258804 0.92022276]
  [0.28297934 0.59827878 0.34790843 ... 0.43747266 0.51005106 0.64974711]
  ...
  [0.57636892 0.36555094 0.6446952  ... 0.29213075 0.76282664 0.49437503]
  [0.40006183 0.74807067 0.99801612 ... 0.65607391 0.61078061 0.8439876 ]
  [0.88862785 0.66311741 0.17490082 ... 0.64284041 0.45705369 0.55345946]]

 [[0.45947162 0.33731738 0.90438631 ... 0.54528598 0.30649124 0.35571294]
  [0.41115774 0.50876686 0.56284005 ... 0.42112742 0.61152903 0.72970364]
  [0.66048007 0.8539097  0.50803717 ... 0.81862748 0.65258954 0.69667883]
  ...
  [0.30553031 0.47874142 0.52453558 ... 0.74887097 0.58714094 0.8366952 ]
  [0.55400727 0.72459279 0.77272089 ... 0.95152889 0.74165556 0.62770609]
  [0.5322632  0.86957058 0.58457017 ... 0.68095686 0.81263984 0.67018678]]

 [[0.54519029 0.20026539 0.48191361 ... 0.83858827 0.45307323 0.18147634]
  [0.53469829 0.491423

/opt/anaconda3/lib/python3.8/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
/opt/anaconda3/lib/python3.8/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
/opt/anaconda3/lib/python3.8/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
/opt/anaconda3/lib/python3.8/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
/opt/anaconda3/lib/python3.8/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
/opt/anaconda3/lib/python3.8/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not mis

In [16]:
#3.6 Concat MI-based dynamic adjacency matrix
#Index all of the rows and columns
DyConcatMI=[]
for k in range(COVID_features_aftslid.shape[0]):
    for i in range(COVID_features_aftslid.shape[2]):
        for j in range(COVID_features_aftslid.shape[2]):
            DyConcatMI.append(Global_MeanMI(COVID_features_aftslid[:k,:,i],
                                    COVID_features_aftslid[:k,:,j],2))
DyConcatMI=np.array(DyConcatMI).reshape(COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[0])
DyConcatMI=DyConcatMI.transpose(2, 0, 1)
from fancyimpute import SimpleFill, NuclearNormMinimization, SoftImpute, BiScaler
for i in range(DyConcatMI.shape[0]):
    MEAN_imputer = SimpleFill()
    DyConcatMI[i,:,:]=MEAN_imputer.fit_transform(DyConcatMI[i,:,:])
print(DyConcatMI)#Here DyConcatMI is done

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/Users/macbook/Desktop/PHD/CIKM/DAMR_CODE/model.py:67: RuntimeWarning: invalid value encountered in true_divide
  c_normalized = c / float(np.sum(c))
/Users/macbook/Desktop/PHD/CIKM/DAMR_CODE/model.py:67: RuntimeWarning: invalid value encountered in true_divide
  c_normalized = c / float(np.sum(c))
/Users/macbook/Desktop/PHD/CIKM/DAMR_CODE/model.py:67: RuntimeWarning: invalid value encountered in true_divide
  c_normalized = c / float(np.sum(c))
/Users/macbook/Desktop/PHD/CIKM/DAMR_CODE/model.py:67: RuntimeWarning: invalid value encountered in true_divide
  c_normalized = c / float(np.sum(c))
/Users/macbook/Desktop/PHD/CIKM/DAMR_CODE/

/Users/macbook/Desktop/PHD/CIKM/DAMR_CODE/model.py:67: RuntimeWarning: invalid value encountered in true_divide
  c_normalized = c / float(np.sum(c))
/Users/macbook/Desktop/PHD/CIKM/DAMR_CODE/model.py:67: RuntimeWarning: invalid value encountered in true_divide
  c_normalized = c / float(np.sum(c))
/Users/macbook/Desktop/PHD/CIKM/DAMR_CODE/model.py:67: RuntimeWarning: invalid value encountered in true_divide
  c_normalized = c / float(np.sum(c))
/Users/macbook/Desktop/PHD/CIKM/DAMR_CODE/model.py:67: RuntimeWarning: invalid value encountered in true_divide
  c_normalized = c / float(np.sum(c))
/Users/macbook/Desktop/PHD/CIKM/DAMR_CODE/model.py:67: RuntimeWarning: invalid value encountered in true_divide
  c_normalized = c / float(np.sum(c))
/Users/macbook/Desktop/PHD/CIKM/DAMR_CODE/model.py:67: RuntimeWarning: invalid value encountered in true_divide
  c_normalized = c / float(np.sum(c))
/Users/macbook/Desktop/PHD/CIKM/DAMR_CODE/model.py:67: RuntimeWarning: invalid value encountered in 

[[[0.57013036 0.91545221 0.57364931 ... 0.41844911 0.24771303 0.26333598]
  [0.70517372 0.55803727 0.48686508 ... 0.30034378 0.29258804 0.92022276]
  [0.28297934 0.59827878 0.34790843 ... 0.43747266 0.51005106 0.64974711]
  ...
  [0.57636892 0.36555094 0.6446952  ... 0.29213075 0.76282664 0.49437503]
  [0.40006183 0.74807067 0.99801612 ... 0.65607391 0.61078061 0.8439876 ]
  [0.88862785 0.66311741 0.17490082 ... 0.64284041 0.45705369 0.55345946]]

 [[0.45947162 0.33731738 0.90438631 ... 0.54528598 0.30649124 0.35571294]
  [0.41115774 0.50876686 0.56284005 ... 0.42112742 0.61152903 0.72970364]
  [0.66048007 0.8539097  0.50803717 ... 0.81862748 0.65258954 0.69667883]
  ...
  [0.30553031 0.47874142 0.52453558 ... 0.74887097 0.58714094 0.8366952 ]
  [0.55400727 0.72459279 0.77272089 ... 0.95152889 0.74165556 0.62770609]
  [0.5322632  0.86957058 0.58457017 ... 0.68095686 0.81263984 0.67018678]]

 [[0.54519029 0.20026539 0.48191361 ... 0.83858827 0.45307323 0.18147634]
  [0.53469829 0.491423

/opt/anaconda3/lib/python3.8/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
/opt/anaconda3/lib/python3.8/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
/opt/anaconda3/lib/python3.8/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
/opt/anaconda3/lib/python3.8/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
/opt/anaconda3/lib/python3.8/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
/opt/anaconda3/lib/python3.8/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not mis

In [17]:
#3.7 FFT-based sum batch's MI dynamic adjacency matrix
print(COVID_features_aftslid.shape)
T=[20,40,100];
Term1=T[0]
Term2=T[1]
Term3=T[2]
#Step1: Calculate Adj in each batch
#(1) Term1
a=[]
from math import ceil#Eg:Ceil(2.2)=3
for i in range(COVID_features_aftslid.shape[2]):
    for j in range(COVID_features_aftslid.shape[2]):
        for k in np.arange(ceil(COVID_features_aftslid.shape[0]/Term1)):
            a.append(Global_MeanMI(COVID_features_aftslid[k*Term1:k*Term1+Term1,:,i],COVID_features_aftslid[k*Term1:k*Term1+Term1,:,j],2))
a=np.array(a).reshape(COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2],ceil(COVID_features_aftslid.shape[0]/Term1))#a:(7, 7, 23)
a=a.transpose(2,0,1)
print(a.shape)#(24,7,7)

#(2) Term2
b=[]
for i in range(COVID_features_aftslid.shape[2]):
    for j in range(COVID_features_aftslid.shape[2]):
        for k in np.arange(ceil(COVID_features_aftslid.shape[0]/Term2)):
            b.append(Global_MeanMI(COVID_features_aftslid[k*Term2:k*Term2+Term2,:,i],COVID_features_aftslid[k*Term2:k*Term2+Term2,:,j],2))
b=np.array(b).reshape(COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2],ceil(COVID_features_aftslid.shape[0]/Term2))#a:(7, 7, 23)
print(b.shape)#(7, 7, 12)
b=b.transpose(2,0,1)
print(b.shape)#(12,7,7)

#(3) Term3
c=[]
for i in range(COVID_features_aftslid.shape[2]):
    for j in range(COVID_features_aftslid.shape[2]):
        for k in np.arange(ceil(COVID_features_aftslid.shape[0]/Term3)):
            c.append(Global_MeanMI(COVID_features_aftslid[k*Term3:k*Term3+Term3,:,i],COVID_features_aftslid[k*Term3:k*Term3+Term3,:,j],2))
c=np.array(c).reshape(COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2],ceil(COVID_features_aftslid.shape[0]/Term3))#a:(7, 7, 23)
print(c.shape)#(7, 7, 5)
c=c.transpose(2,0,1)
print(c.shape)#(5,7,7)
#Step2:Assign it to particular position:
#Each timestamp from the same batch share the same Adj.
#MeanMI
data=[]
for j in range(a.shape[0]):
    for i in range(Term1):
        data.append(a[j,:,:])
data=np.array(data)
test=data
DyConcatMI1=test[:DyConcatMI.shape[0],:,:]
print(DyConcatMI1.shape)#(473,7,7)

data=[]
for j in range(b.shape[0]):
    for i in range(Term2):
        data.append(b[j,:,:])
data=np.array(data)
test=data
DyConcatMI2=test[:DyConcatMI.shape[0],:,:]
print(DyConcatMI2.shape)#(473,7,7)

data=[]
for j in range(c.shape[0]):
    for i in range(Term3):
        data.append(c[j,:,:])
data=np.array(data)
test=data
DyConcatMI3=test[:DyConcatMI.shape[0],:,:]
print(DyConcatMI3.shape)#(473,7,7)

# Step3: Add all of the adjacency matrices
DyConcatFFTMI=[]
DyConcatFFTMI=DyConcatMI1+DyConcatMI2+DyConcatMI3
print(DyConcatFFTMI.shape)#(473,7,7)

# Step4: Normalize adjacency matrices
from sklearn.preprocessing import MinMaxScaler
for i in range(DyConcatFFTMI.shape[0]):
    scaler = MinMaxScaler(feature_range=(0, 1))
    DyConcatFFTMI[i,:,:] = scaler.fit_transform(DyConcatFFTMI[i,:,:])
print(DyConcatFFTMI)

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(473, 3, 7)
(24, 7, 7)
(7, 7, 12)
(12, 7, 7)
(7, 7, 5)
(5, 7, 7)
(473, 7, 7)
(473, 7, 7)
(473, 7, 7)
(473, 7, 7)
[[[1.         0.14312873 0.01090966 ... 0.19109867 0.10525115 0.28307437]
  [0.         1.         0.         ... 0.         0.         0.        ]
  [0.21827035 0.40009327 1.         ... 0.13508498 0.28293916 0.32002389]
  ...
  [0.31974433 0.35193694 0.0932996  ... 1.         0.15601411 0.28622958]
  [0.18132473 0.27023703 0.14364186 ... 0.06095244 1.         0.24694028]
  [0.17773889 0.         0.00803653 ... 0.01272219 0.04897924 1.        ]]

 [[1.         0.14312873 0.01090966 ... 0.19109867 0.10525115 0.28307437]
  [0.         1.         0.         ... 0.         0.         0.        ]
  [0.21827035 0.40009327 1.         ... 0.13508498 0.28293916 0.32002389]
  ...
  [0.31974433 0.35193694 0.0932996  ... 1.         0.15601411 0.28622958]
  [0.18132473 0.27023703 0.14364186 ... 0.06095244 1.         0.24694028]
  [0.17773889 0.         0.00803653 ... 0.01272219 0.048979

In [18]:
#3.8 FFT-based MI-based dynamic adjacency matrix
print(COVID_features_aftslid.shape)
T=[20,40,100];
Term1=T[0]
Term2=T[1]
Term3=T[2]

#Step1: Calculate Adj in each batch

#(1) Term1
a=[]
from math import ceil#Eg:Ceil(2.2)=3
for i in range(COVID_features_aftslid.shape[2]):
    for j in range(COVID_features_aftslid.shape[2]):
        for k in np.arange(ceil(COVID_features_aftslid.shape[0]/Term1)):
            a.append(Global_MeanMI(COVID_features_aftslid[k*Term1:k*Term1+Term1,:,i],COVID_features_aftslid[k*Term1:k*Term1+Term1,:,j],2))
a=np.array(a).reshape(COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2],ceil(COVID_features_aftslid.shape[0]/Term1))#a:(7, 7, 23)
a=a.transpose(2,0,1)
print(a.shape)#(24,7,7)

#(2) Term2
b=[]
for i in range(COVID_features_aftslid.shape[2]):
    for j in range(COVID_features_aftslid.shape[2]):
        for k in np.arange(ceil(COVID_features_aftslid.shape[0]/Term2)):
            b.append(Global_MeanMI(COVID_features_aftslid[k*Term2:k*Term2+Term2,:,i],COVID_features_aftslid[k*Term2:k*Term2+Term2,:,j],2))
b=np.array(b).reshape(COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2],ceil(COVID_features_aftslid.shape[0]/Term2))#a:(7, 7, 23)
print(b.shape)#(7, 7, 12)
b=b.transpose(2,0,1)
print(b.shape)#(12,7,7)

#(3) Term3
c=[]
for i in range(COVID_features_aftslid.shape[2]):
    for j in range(COVID_features_aftslid.shape[2]):
        for k in np.arange(ceil(COVID_features_aftslid.shape[0]/Term3)):
            c.append(Global_MeanMI(COVID_features_aftslid[k*Term3:k*Term3+Term3,:,i],COVID_features_aftslid[k*Term3:k*Term3+Term3,:,j],2))
c=np.array(c).reshape(COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2],ceil(COVID_features_aftslid.shape[0]/Term3))#a:(7, 7, 23)
print(c.shape)#(7, 7, 5)
c=c.transpose(2,0,1)
print(c.shape)#(5,7,7)
#Step2: Calculate MI's MI for periodic term 1
#Term1
temp=[]
for j in range(a.shape[0]):
    for i in range(a.shape[0]):
        temp.append(Global_MeanMI(a[j,:,:],a[i,:,:],2))#The first point MI's MI 
temp=np.array(temp).reshape(a.shape[0],a.shape[0])#24*24

#Step3: Weighted sum for the same periodic term
index_all=np.arange(temp.shape[0]*temp.shape[0]).reshape(a.shape[0],a.shape[0])#Initialization of index_all,shape=24*24
for i in range(temp.shape[0]):
    arrIndex=temp[i,:].argsort()
    temp[i,:]=temp[i,:][arrIndex[::-1]]
    index_all[i,:]=arrIndex
print(temp.shape)#24*24
print(index_all.shape)#24*24

all_term=5#Get the first 5 decomposition
a_all=np.zeros((a.shape[0],COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2]))
for i in range(a.shape[0]):
    for u in range(all_term):
        a_all[i,:,:]=a_all[i,:,:]+a[i,:,:]+index_all[i,u]*temp[i,u]
print(a_all.shape)#(24,7,7)

#Step2: Calculate MI's MI for periodic term 2
#Term2
temp=[]
for j in range(b.shape[0]):
    for i in range(b.shape[0]):
        temp.append(Global_MeanMI(b[j,:,:],b[i,:,:],2))#The first point MI's MI 
temp=np.array(temp).reshape(b.shape[0],b.shape[0])#12*12

#Step3: Weighted sum for the same periodic term
index_all=np.arange(temp.shape[0]*temp.shape[0]).reshape(b.shape[0],b.shape[0])#Initialization of index_all,shape=24*24
for i in range(temp.shape[0]):
    arrIndex=temp[i,:].argsort()
    temp[i,:]=temp[i,:][arrIndex[::-1]]
    index_all[i,:]=arrIndex
print(temp.shape)#12*12
print(index_all.shape)#12*12

all_term=5#Get the first 5 decomposition
b_all=np.zeros((b.shape[0],COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2]))
for i in range(b.shape[0]):
    for u in range(all_term):
        b_all[i,:,:]=b_all[i,:,:]+b[i,:,:]+index_all[i,u]*temp[i,u]
print(b_all.shape)#(12,7,7)

#Step2: Calculate MI's MI for periodic term 3
#Term3
temp=[]
for j in range(c.shape[0]):
    for i in range(c.shape[0]):
        temp.append(Global_MeanMI(c[j,:,:],c[i,:,:],2))#The first point MI's MI 
temp=np.array(temp).reshape(c.shape[0],c.shape[0])#12*12

#Step3: Weighted sum for the same periodic term
index_all=np.arange(temp.shape[0]*temp.shape[0]).reshape(c.shape[0],c.shape[0])#Initialization of index_all,shape=24*24
for i in range(temp.shape[0]):
    arrIndex=temp[i,:].argsort()
    temp[i,:]=temp[i,:][arrIndex[::-1]]
    index_all[i,:]=arrIndex
print(temp.shape)#5*5
print(index_all.shape)#5*5

all_term=5#Get the first 5 decomposition
c_all=np.zeros((c.shape[0],COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2]))
for i in range(c.shape[0]):
    for u in range(all_term):
        c_all[i,:,:]=c_all[i,:,:]+c[i,:,:]+index_all[i,u]*temp[i,u]
print(c_all.shape)#(5,7,7)

#Step4:Assign it to particular position:
#Each timestamp from the same batch share the same Adj.
#MeanMI
data=[]
for j in range(a_all.shape[0]):
    for i in range(Term1):
        data.append(a_all[j,:,:])
data=np.array(data)
test=data
DyConcatMI1=test[:DyConcatMI.shape[0],:,:]
print(DyConcatMI1.shape)#(473,7,7)

data=[]
for j in range(b_all.shape[0]):
    for i in range(Term2):
        data.append(b_all[j,:,:])
data=np.array(data)
test=data
DyConcatMI2=test[:DyConcatMI.shape[0],:,:]
print(DyConcatMI2.shape)#(473,7,7)

data=[]
for j in range(c_all.shape[0]):
    for i in range(Term3):
        data.append(c_all[j,:,:])
data=np.array(data)
test=data
DyConcatMI3=test[:DyConcatMI.shape[0],:,:]
print(DyConcatMI3.shape)#(473,7,7)

# Step5: Add all of the adjacency matrices
DyConcatFFTeachMI=DyConcatMI1+DyConcatMI2+DyConcatMI3
print(DyConcatFFTeachMI.shape)#(473,7,7)

# Step6: Normalize adjacency matrices
from sklearn.preprocessing import MinMaxScaler
for i in range(DyConcatFFTeachMI.shape[0]):
    scaler = MinMaxScaler(feature_range=(0, 1))
    DyConcatFFTeachMI[i,:,:] = scaler.fit_transform(DyConcatFFTeachMI[i,:,:])
print(DyConcatFFTeachMI)


/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(473, 3, 7)
(24, 7, 7)
(7, 7, 12)
(12, 7, 7)
(7, 7, 5)
(5, 7, 7)
(24, 24)
(24, 24)
(24, 7, 7)
(12, 12)
(12, 12)
(12, 7, 7)
(5, 5)
(5, 5)
(5, 7, 7)
(473, 7, 7)
(473, 7, 7)
(473, 7, 7)
(473, 7, 7)
[[[1.         0.14312873 0.01090966 ... 0.19109867 0.10525115 0.28307437]
  [0.         1.         0.         ... 0.         0.         0.        ]
  [0.21827035 0.40009327 1.         ... 0.13508498 0.28293916 0.32002389]
  ...
  [0.31974433 0.35193694 0.0932996  ... 1.         0.15601411 0.28622958]
  [0.18132473 0.27023703 0.14364186 ... 0.06095244 1.         0.24694028]
  [0.17773889 0.         0.00803653 ... 0.01272219 0.04897924 1.        ]]

 [[1.         0.14312873 0.01090966 ... 0.19109867 0.10525115 0.28307437]
  [0.         1.         0.         ... 0.         0.         0.        ]
  [0.21827035 0.40009327 1.         ... 0.13508498 0.28293916 0.32002389]
  ...
  [0.31974433 0.35193694 0.0932996  ... 1.         0.15601411 0.28622958]
  [0.18132473 0.27023703 0.14364186 ... 0.06095244 

# Begin with DAMR

In [19]:
##Begin with DAMR
print(COVID_features_aftslid.shape)#473*3*7
print(COVID_X1_all_aftslid.shape)#473*7*30
print(COVID_Y1_all.shape)#(473, 7, 1)
print(Adj_dist.shape)#7*7
# Set Hyperpaprameters
tf.keras.backend.clear_session()#clear all
tf.autograph.set_verbosity(0)#reset to zero
learning_rate = 0.0001
batch_size =24
epochs = 100
seed = 42
verbose = 1

(473, 3, 7)
(473, 7, 30)
(473, 7, 1)
(7, 7)


/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [20]:
## Define Model---GAT+GCN+DIFFCONV
#inputs_ret = Input(shape=(COVID_X1_all_aftslid.shape[1], COVID_X1_all_aftslid.shape[2], ))#7*30=node*lags(Time to look back)
inputs_feat = Input(shape=(COVID_features_aftslid.shape[2], COVID_features_aftslid.shape[1], ))#7*6=Node*feature
inputs_adj = Input(shape=(Adj_dist.shape[1],Adj_dist.shape[1], ))#7*7=node*node
#print(inputs_ret.shape)
print(inputs_feat.shape)
print(inputs_adj.shape)

(None, 7, 3)
(None, 7, 7)


/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [21]:
#GATinput:1.input nodes*lags 2.adj:nodes*nodes
#GAToutput:1.channels*att_heads2.Att_weights=nodes*heads*nodes
# GAT_output_P, Att_weights_P =  GATConv(16, attn_heads=4, concat_heads=True, dropout_rate=0.3, return_attn_coef=True, 
#                                    activation='relu', use_bias=False)([inputs_ret, inputs_adj])

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [22]:
# Att_weights_P = tf.math.reduce_mean(Att_weights_P, axis=2)#reduce heads,it will be nodes*nodes
# print(Att_weights_P)

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [23]:
# GCN_output_P =  GCNConv(16, dropout_rate=0.3, 
#                         activation='relu', use_bias=True)([inputs_feat, inputs_adj])

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [24]:
diff_out = DiffusionConv(16, K=6, activation='tanh')([inputs_feat, inputs_adj])#diff_out=nodes*channels

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [25]:
# con_out_P_2 = Concatenate(axis=-1)([GAT_output_P, GCN_output_P,diff_out])
dense_P = Dense(8, activation='relu')(diff_out)

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [26]:
dense_out = Dropout(0.2)(dense_P)
outputs = Dense(1)(dense_out)

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [27]:
model = Model(inputs=[inputs_feat, inputs_adj], outputs=outputs)
model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse', metrics=["mae", "mape"])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 7, 3)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 7, 7)]       0                                            
__________________________________________________________________________________________________
diffusion_conv (DiffusionConv)  (None, 7, 16)        112         input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
dense (Dense)                   (None, 7, 8)         136         diffusion_conv[0][0]         

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [28]:
#Get 473*7*7 for static adjacency matrix
#3.1 Adj_dist
Adj_dist=adj_dist(COVID_lat,COVID_long)#7*7
Adj_dist=pd.DataFrame(Adj_dist)
Adj_all = np.arange(473*7*7).reshape(473,7,7)#(473,7,7)
Adj_all[0,:,:]=pd.DataFrame(Adj_all[0,:,:])
Adj_all[0,:,:]=Adj_dist
data=[]
for i in range(Adj_all.shape[0]):
    data.append(Adj_dist)
data=np.array(data)
Adj_dist=data
#3.2 MeanMI
Adj_MeanMI=MeanMI
Adj_all = np.arange(473*7*7).reshape(473,7,7)#(473,7,7)
Adj_all[0,:,:]=pd.DataFrame(Adj_all[0,:,:])
Adj_all[0,:,:]=MeanMI
data=[]
for i in range(Adj_all.shape[0]):
    data.append(Adj_MeanMI)
data=np.array(data)
Adj_MeanMI=data
#3.3 ConcatMI
Adj_ConcatMI=ConcatMI
Adj_all = np.arange(473*7*7).reshape(473,7,7)#(473,7,7)
Adj_all[0,:,:]=pd.DataFrame(Adj_all[0,:,:])
Adj_all[0,:,:]=Adj_ConcatMI
data=[]
for i in range(Adj_all.shape[0]):
    data.append(Adj_ConcatMI)
data=np.array(data)
Adj_ConcatMI=data
#3.4 FeatureMI
Adj_FeatureMI=FeatureMI
Adj_all = np.arange(473*7*7).reshape(473,7,7)#(473,7,7)
Adj_all[0,:,:]=pd.DataFrame(Adj_all[0,:,:])
Adj_all[0,:,:]=Adj_FeatureMI
data=[]
for i in range(Adj_all.shape[0]):
    data.append(Adj_FeatureMI)
data=np.array(data)
Adj_FeatureMI=data
# #3.9 ->MeanMI approach(3.1+3.2+3.5+3.8)
# coe1=0.1
# coe2=0.8
# coe3=0.4
# coe4=0.1
# Adj_all_MeanMI=coe1*Adj_dist+coe2*Adj_MeanMI+coe3*DyMeanMI+coe4*DyConcatFFTeachMI
# scaler = MinMaxScaler(feature_range=(-1, 1))
# for i in range(Adj_all_MeanMI.shape[0]):
#     Adj_all_MeanMI[i,:,:]=scaler.fit_transform(Adj_all_MeanMI[i,:,:])
# #3.10 ->()ConcatMI approach
# Adj_all_ConcatMI=coe1*Adj_dist+coe2*Adj_MeanMI+coe3*DyMeanMI+coe4*DyConcatFFTeachMI
# for i in range(Adj_all_ConcatMI.shape[0]):
#     Adj_all_ConcatMI[i,:,:]=scaler.fit_transform(Adj_all_ConcatMI[i,:,:])

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [29]:
scaler = MinMaxScaler(feature_range=(0, 1))

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [30]:
# #DyMeanMI
# #3.2 Mean MI-based dynamic adjacency matrix
# print(COVID_features_aftslid.shape)#(473, 3, 7)
# #Index all of the rows and columns
# MeanMI=[]
# for k in range(COVID_features_aftslid.shape[0]):
#     for i in range(COVID_features_aftslid.shape[2]):
#         for j in range(COVID_features_aftslid.shape[2]):
#             MeanMI.append(Global_MeanMI(COVID_features_aftslid[:k,:,i],
#                                     COVID_features_aftslid[:k,:,j],2))
# DyMeanMI=np.array(MeanMI).reshape(COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[2],COVID_features_aftslid.shape[0])
# DyMeanMI=DyMeanMI.transpose(2, 0, 1)
# print(DyMeanMI.shape)#(473,7,7)

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [31]:

print(COVID_features_aftslid.shape)#473*3*7
COVID_features_aftslid= np.transpose(COVID_features_aftslid,(0,2,1))#473*7*3
print(COVID_X1_all_aftslid.shape)#473*7*30
print(np.array(COVID_Y1_all).shape)#(473, 7, 1)
print(np.array(Adj_MeanMI).shape)#473*7*7
#Normalization
scaler = MinMaxScaler(feature_range=(-1, 1))
scaled1 = scaler.fit_transform(COVID_Y1_all[:,:,0])
COVID_Y1_all[:,:,0]=scaled1


/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(473, 3, 7)
(473, 7, 30)
(473, 7, 1)
(473, 7, 7)


In [32]:
tf.keras.backend.clear_session()#clear all
tf.autograph.set_verbosity(0)#reset to zero
learning_rate = 0.0001
batch_size =72
epochs = 100
seed = 42
verbose = 1
ChangeAdj=Adj_FeatureMI
x_train=COVID_X1_all_aftslid[:int(len(COVID_X1_all_aftslid)*0.8),:,:]
x_val=COVID_X1_all_aftslid[int(len(COVID_X1_all_aftslid)*0.8):int(len(COVID_X1_all_aftslid)*0.9),:,:]
x_test=COVID_X1_all_aftslid[int(len(COVID_X1_all_aftslid)*0.9):,:,:]
features_train=COVID_features_aftslid[:int(len(COVID_features_aftslid)*0.8),:,:]
features_val=COVID_features_aftslid[int(len(COVID_features_aftslid)*0.8):int(len(COVID_features_aftslid)*0.9),:,:]
features_test=COVID_features_aftslid[int(len(COVID_features_aftslid)*0.9):,:,:]
adj_train=ChangeAdj[:int(len(ChangeAdj)*0.8),:,:]
adj_val=ChangeAdj[int(len(ChangeAdj)*0.8):int(len(ChangeAdj)*0.9),:,:]
adj_test=ChangeAdj[int(len(ChangeAdj)*0.9):,:,:]
y_train=COVID_Y1_all[:int(len(COVID_Y1_all)*0.8),:,:]
y_val=COVID_Y1_all[int(len(COVID_Y1_all)*0.8):int(len(COVID_Y1_all)*0.9),:,:]
y_test=COVID_Y1_all[int(len(COVID_Y1_all)*0.9):,:,:]

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [33]:
print(x_train.shape)
print(features_train.shape)
print(adj_train.shape)
print(y_train.shape)

(378, 7, 30)
(378, 7, 3)
(378, 7, 7)
(378, 7, 1)


/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [34]:
# features_train= np.transpose(features_train,(0,2,1))
# features_val= np.transpose(features_val,(0,2,1))
# features_test= np.transpose(features_test,(0,2,1))

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [35]:
tf.config.experimental_run_functions_eagerly(True)
# Set Hyperpaprameters
tf.keras.backend.clear_session()#clear all
tf.autograph.set_verbosity(0)#reset to zero


# hists = model.fit(x= [x_train, features_train, adj_train],
#                   y= y_train, verbose=verbose, epochs=500,
#                   batch_size=batch_size, #validation_split=0.1, 
#                   validation_data=([x_val, features_val, adj_val], y_val),
#                   #test_data=([x_test, features_test, adj_test], y_test),
#   )

##If you want to train faster, try earlystopping.

hists = model.fit(x= [features_train, adj_train],
                  y= y_train, verbose=verbose, epochs=500,
                  batch_size=batch_size, #validation_split=0.1, 
                  validation_data=([features_val, adj_val], y_val),
                  #test_data=([x_test, features_test, adj_test], y_test),
                    callbacks=[EarlyStopping(#monitor="val_mean_absolute_error", 
                      monitor="val_loss", patience=20, restore_best_weights=True)],
  )


Instructions for updating:
Use `tf.config.run_functions_eagerly` instead of the experimental version.


/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/data/ops/dataset_ops.py:4211: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/data/ops/dataset_ops.py:4211: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf

Epoch 1/500
6/6 [==============================] - 1s 128ms/step - loss: 7.7106 - mae: 2.5120 - mape: 1109.2670 - val_loss: 4.6517 - val_mae: 2.1332 - val_mape: 567.2863
Epoch 2/500


/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/data/ops/dataset_ops.py:4211: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/data/ops/dataset_ops.py:4211: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/data/ops/dataset_ops.py:4211: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.ex

6/6 [==============================] - 1s 117ms/step - loss: 7.4700 - mae: 2.4849 - mape: 1015.0900 - val_loss: 4.5297 - val_mae: 2.1051 - val_mape: 560.6058
Epoch 3/500
6/6 [==============================] - 1s 111ms/step - loss: 7.5399 - mae: 2.4906 - mape: 976.4321 - val_loss: 4.4097 - val_mae: 2.0771 - val_mape: 553.9556
Epoch 4/500
6/6 [==============================] - 1s 129ms/step - loss: 7.2517 - mae: 2.4381 - mape: 1029.6409 - val_loss: 4.2927 - val_mae: 2.0495 - val_mape: 547.4258
Epoch 5/500
6/6 [==============================] - 1s 114ms/step - loss: 7.0537 - mae: 2.4043 - mape: 1039.2845 - val_loss: 4.1766 - val_mae: 2.0217 - val_mape: 540.8434
Epoch 6/500
6/6 [==============================] - 1s 106ms/step - loss: 6.8273 - mae: 2.3688 - mape: 904.5183 - val_loss: 4.0602 - val_mae: 1.9934 - val_mape: 534.1255
Epoch 7/500
6/6 [==============================] - 1s 121ms/step - loss: 6.7009 - mae: 2.3471 - mape: 901.0131 - val_loss: 3.9495 - val_mae: 1.9661 - val_mape: 527.

...

6/6 [==============================] - 0s 59ms/step - loss: 0.0951 - mae: 0.2444 - mape: 157.1539 - val_loss: 0.3192 - val_mae: 0.5351 - val_mape: 117.9918
Epoch 291/500
6/6 [==============================] - 0s 61ms/step - loss: 0.0958 - mae: 0.2447 - mape: 150.2083 - val_loss: 0.3189 - val_mae: 0.5347 - val_mape: 117.8040
Epoch 292/500
6/6 [==============================] - 0s 58ms/step - loss: 0.0928 - mae: 0.2406 - mape: 150.9715 - val_loss: 0.3184 - val_mae: 0.5343 - val_mape: 117.6464
Epoch 293/500
6/6 [==============================] - 0s 58ms/step - loss: 0.0965 - mae: 0.2450 - mape: 172.2149 - val_loss: 0.3180 - val_mae: 0.5338 - val_mape: 117.4664
Epoch 294/500
6/6 [==============================] - 0s 67ms/step - loss: 0.0939 - mae: 0.2412 - mape: 136.6003 - val_loss: 0.3176 - val_mae: 0.5334 - val_mape: 117.2756
Epoch 295/500
6/6 [==============================] - 0s 59ms/step - loss: 0.0972 - mae: 0.2431 - mape: 146.6557 - val_loss: 0.3171 - val_mae: 0.5330 - val_mape: 117

6/6 [==============================] - 0s 59ms/step - loss: 0.0775 - mae: 0.2193 - mape: 153.5044 - val_loss: 0.2938 - val_mae: 0.5096 - val_mape: 109.7946
Epoch 339/500
6/6 [==============================] - 0s 58ms/step - loss: 0.0812 - mae: 0.2211 - mape: 148.2230 - val_loss: 0.2931 - val_mae: 0.5089 - val_mape: 109.6165
Epoch 340/500
6/6 [==============================] - 0s 66ms/step - loss: 0.0771 - mae: 0.2183 - mape: 157.2430 - val_loss: 0.2924 - val_mae: 0.5083 - val_mape: 109.4622
Epoch 341/500
6/6 [==============================] - 0s 55ms/step - loss: 0.0846 - mae: 0.2276 - mape: 132.4480 - val_loss: 0.2919 - val_mae: 0.5077 - val_mape: 109.3464
Epoch 342/500
6/6 [==============================] - 0s 56ms/step - loss: 0.0806 - mae: 0.2214 - mape: 175.8894 - val_loss: 0.2913 - val_mae: 0.5071 - val_mape: 109.2204
Epoch 343/500
6/6 [==============================] - 0s 59ms/step - loss: 0.0797 - mae: 0.2215 - mape: 153.2345 - val_loss: 0.2906 - val_mae: 0.5065 - val_mape: 109

6/6 [==============================] - 0s 63ms/step - loss: 0.0737 - mae: 0.2115 - mape: 152.5840 - val_loss: 0.2650 - val_mae: 0.4792 - val_mape: 101.9786
Epoch 387/500
6/6 [==============================] - 0s 57ms/step - loss: 0.0693 - mae: 0.2052 - mape: 165.9159 - val_loss: 0.2647 - val_mae: 0.4788 - val_mape: 101.8607
Epoch 388/500
6/6 [==============================] - 0s 60ms/step - loss: 0.0714 - mae: 0.2094 - mape: 118.8566 - val_loss: 0.2642 - val_mae: 0.4782 - val_mape: 101.7045
Epoch 389/500
6/6 [==============================] - 0s 57ms/step - loss: 0.0681 - mae: 0.2028 - mape: 142.5293 - val_loss: 0.2638 - val_mae: 0.4777 - val_mape: 101.5831
Epoch 390/500
6/6 [==============================] - 0s 57ms/step - loss: 0.0694 - mae: 0.2052 - mape: 118.9789 - val_loss: 0.2635 - val_mae: 0.4774 - val_mape: 101.4957
Epoch 391/500
6/6 [==============================] - 0s 65ms/step - loss: 0.0705 - mae: 0.2066 - mape: 148.8765 - val_loss: 0.2630 - val_mae: 0.4769 - val_mape: 101

6/6 [==============================] - 0s 64ms/step - loss: 0.0608 - mae: 0.1936 - mape: 146.3873 - val_loss: 0.2372 - val_mae: 0.4474 - val_mape: 94.1061
Epoch 435/500
6/6 [==============================] - 0s 61ms/step - loss: 0.0649 - mae: 0.2000 - mape: 140.5757 - val_loss: 0.2368 - val_mae: 0.4468 - val_mape: 93.9814
Epoch 436/500
6/6 [==============================] - 0s 62ms/step - loss: 0.0639 - mae: 0.1974 - mape: 145.2509 - val_loss: 0.2365 - val_mae: 0.4465 - val_mape: 93.9087
Epoch 437/500
6/6 [==============================] - 0s 65ms/step - loss: 0.0607 - mae: 0.1931 - mape: 146.9409 - val_loss: 0.2361 - val_mae: 0.4460 - val_mape: 93.7595
Epoch 438/500
6/6 [==============================] - 0s 62ms/step - loss: 0.0610 - mae: 0.1921 - mape: 118.4407 - val_loss: 0.2356 - val_mae: 0.4454 - val_mape: 93.5860
Epoch 439/500
6/6 [==============================] - 0s 65ms/step - loss: 0.0567 - mae: 0.1867 - mape: 179.9147 - val_loss: 0.2351 - val_mae: 0.4448 - val_mape: 93.4189


6/6 [==============================] - 0s 60ms/step - loss: 0.0558 - mae: 0.1849 - mape: 138.7140 - val_loss: 0.2144 - val_mae: 0.4197 - val_mape: 87.8436
Epoch 483/500
6/6 [==============================] - 0s 65ms/step - loss: 0.0566 - mae: 0.1863 - mape: 140.4760 - val_loss: 0.2136 - val_mae: 0.4188 - val_mape: 87.6694
Epoch 484/500
6/6 [==============================] - 0s 62ms/step - loss: 0.0547 - mae: 0.1833 - mape: 138.8587 - val_loss: 0.2132 - val_mae: 0.4182 - val_mape: 87.5818
Epoch 485/500
6/6 [==============================] - 0s 65ms/step - loss: 0.0554 - mae: 0.1850 - mape: 135.3569 - val_loss: 0.2127 - val_mae: 0.4176 - val_mape: 87.4509
Epoch 486/500
6/6 [==============================] - 0s 66ms/step - loss: 0.0559 - mae: 0.1851 - mape: 207.7254 - val_loss: 0.2122 - val_mae: 0.4170 - val_mape: 87.2811
Epoch 487/500
6/6 [==============================] - 0s 62ms/step - loss: 0.0550 - mae: 0.1828 - mape: 132.4498 - val_loss: 0.2117 - val_mae: 0.4164 - val_mape: 87.1373


In [36]:
def mae(y_true, y_pred):
    return np.mean(np.abs(y_pred - y_true))

def rmse(y_true, y_pred):
    return np.sqrt(np.mean(np.square(y_pred - y_true)))

def mape(y_true, y_pred, threshold=0.1):
    v = np.clip(np.abs(y_true), threshold, None)
    diff = np.abs((y_true - y_pred) / v)
    return np.mean(diff, axis=-1).mean()
###Write the function of mre
def mre(y_true, y_pred):
    return np.sum(np.abs(y_pred - y_true))/np.sum(np.abs(y_true))
def get_metrics(y, yp):
    return {

        "mae": np.round(mae(y, yp), 4),
        "mape": np.round(mape(y, yp),4),
        "mre": np.round(mre(y, yp),4),
        "rmse": np.round(rmse(y, yp), 4),
        #"MPE": np.round(MPE(y, yp), 4),
        #"R2": np.round(R_squared(y, yp),4)   
    }

# scaled1 = scaler.fit_transform(MeanMI)
#Get the tensor of test set after masking it.
test_predicted = model.predict([features_test, adj_test])
get_metrics(y_test.flatten(), test_predicted.flatten())


/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/data/ops/dataset_ops.py:4211: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/data/ops/dataset_ops.py:4211: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf

{'mae': 0.7282, 'mape': 0.8406, 'mre': 0.8495, 'rmse': 0.7469}

In [37]:
#Get the mask of y_test.
Mask_BZ=pd.read_csv("Data/COVID/10%/BZ_10.csv")
Mask_CL=pd.read_csv("Data/COVID/10%/CL_10.csv")
Mask_ID=pd.read_csv("Data/COVID/10%/ID_10.csv")
Mask_IR=pd.read_csv("Data/COVID/10%/IR_10.csv")
Mask_MX=pd.read_csv("Data/COVID/10%/MX_10.csv")
Mask_PR=pd.read_csv("Data/COVID/10%/PR_10.csv")
Mask_SA=pd.read_csv("Data/COVID/10%/SA_10.csv")

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [38]:
Mask_BZ=np.array(Mask_BZ)
missing_mask=[]
for i in range(Mask_BZ.shape[0]):
    missing_mask.append(np.isnan(np.array(Mask_BZ[i])))
missing_mask=np.array(missing_mask).reshape(Mask_BZ.shape[0],Mask_BZ.shape[1])
missing_mask_BZ_test=missing_mask[-15-y_test.shape[0]:-15,:]#48*3
#Get the first feature's mask of BZ
missing_mask_BZ_test1=missing_mask_BZ_test[:,0]

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [39]:
Mask_CL=np.array(Mask_CL)
missing_mask=[]
for i in range(Mask_CL.shape[0]):
    missing_mask.append(np.isnan(np.array(Mask_CL[i])))
missing_mask=np.array(missing_mask).reshape(Mask_CL.shape[0],Mask_CL.shape[1])
missing_mask_CL_test=missing_mask[-15-y_test.shape[0]:-15,:]#48*3
#Get the first feature's mask of CL
missing_mask_CL_test1=missing_mask_CL_test[:,0]
Mask_ID=np.array(Mask_ID)
missing_mask=[]
for i in range(Mask_ID.shape[0]):
    missing_mask.append(np.isnan(np.array(Mask_ID[i])))
missing_mask=np.array(missing_mask).reshape(Mask_ID.shape[0],Mask_ID.shape[1])
missing_mask_ID_test=missing_mask[-15-y_test.shape[0]:-15,:]#48*3
#Get the first feature's mask of ID
missing_mask_ID_test1=missing_mask_ID_test[:,0]
Mask_IR=np.array(Mask_IR)
missing_mask=[]
for i in range(Mask_IR.shape[0]):
    missing_mask.append(np.isnan(np.array(Mask_IR[i])))
missing_mask=np.array(missing_mask).reshape(Mask_IR.shape[0],Mask_IR.shape[1])
missing_mask_IR_test=missing_mask[-15-y_test.shape[0]:-15,:]#48*3
#Get the first feature's mask of IR
missing_mask_IR_test1=missing_mask_IR_test[:,0]
Mask_MX=np.array(Mask_MX)
missing_mask=[]
for i in range(Mask_MX.shape[0]):
    missing_mask.append(np.isnan(np.array(Mask_MX[i])))
missing_mask=np.array(missing_mask).reshape(Mask_MX.shape[0],Mask_MX.shape[1])
missing_mask_MX_test=missing_mask[-15-y_test.shape[0]:-15,:]#48*3
#Get the first feature's mask of MX
missing_mask_MX_test1=missing_mask_MX_test[:,0]
Mask_PR=np.array(Mask_PR)
missing_mask=[]
for i in range(Mask_PR.shape[0]):
    missing_mask.append(np.isnan(np.array(Mask_PR[i])))
missing_mask=np.array(missing_mask).reshape(Mask_PR.shape[0],Mask_PR.shape[1])
missing_mask_PR_test=missing_mask[-15-y_test.shape[0]:-15,:]#48*3
#Get the first feature's mask of PR
missing_mask_PR_test1=missing_mask_PR_test[:,0]
Mask_SA=np.array(Mask_SA)
missing_mask=[]
for i in range(Mask_SA.shape[0]):
    missing_mask.append(np.isnan(np.array(Mask_SA[i])))
missing_mask=np.array(missing_mask).reshape(Mask_SA.shape[0],Mask_SA.shape[1])
missing_mask_SA_test=missing_mask[-15-y_test.shape[0]:-15,:]#48*3
#Get the first feature's mask of SA
missing_mask_SA_test1=missing_mask_SA_test[:,0]

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [40]:
missing_mask_ALL_test1=np.stack((missing_mask_BZ_test1,missing_mask_CL_test1
            ,missing_mask_ID_test1,missing_mask_IR_test1,
            missing_mask_MX_test1,missing_mask_PR_test1,missing_mask_SA_test1
           ),axis=1)

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [41]:
#Get test set result.
get_metrics(y_test[missing_mask_ALL_test1].flatten(), test_predicted[missing_mask_ALL_test1].flatten())

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{'mae': 0.7182, 'mape': 0.8277, 'mre': 0.8399, 'rmse': 0.7392}